In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd drive/MyDrive/test-paper-detector/detectron2

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt

from PIL import Image

In [ ]:
!pip install tf_slim

In [4]:
import cv2
from operator import itemgetter

def draw_bounding_boxes(img, output_dict,count):
    boxlist = []
    height, width, _ = img.shape
 
    obj_index = output_dict['detection_scores'] > 0.8
    
    scores = output_dict['detection_scores'][obj_index]
    boxes = output_dict['detection_boxes'][obj_index]
    classes = output_dict['detection_classes'][obj_index]

    for i in range(len(boxes)):
      boxlist.append([boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]])
    
    boxlist.sort(key=lambda x:x[1])
    print(boxlist)

    if(len(boxlist) >= 3):
      if(len(boxlist) == 4):
        if(boxlist[0][0] > boxlist[1][0]):
          boxlist[0], boxlist[1] = boxlist[1], boxlist[0]

        if(boxlist[2][0] > boxlist[3][0]):
          boxlist[2], boxlist[3] = boxlist[3], boxlist[2]
      else:
        if(boxlist[0][0] > boxlist[1][0]):
          boxlist[0], boxlist[1] = boxlist[1], boxlist[0]
    print(boxlist)
 
    count1 = 0
    for box in boxlist:
        count1 += 1

        cut_img = img[int(box[0] * height):int(box[2] * height), int(box[1] * width):int(box[3] * width)].copy()
        save_img = Image.fromarray(cut_img)
        save_img.save("data/images_choice/train/cut_jpgs/image_{0}{1}.jpg".format(count,count1))
        
    return img

In [ ]:
import os
import tensorflow as tf

PATH_TO_FROZEN_GRAPH = 'research/inference_graph/frozen_inference_graph.pb'
 
detection_graph = tf.Graph()
with detection_graph.as_default():
 
    od_graph_def = tf.compat.v1.GraphDef()
 
    with tf.compat.v2.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as f:
 
        serialized_graph = f.read()
        od_graph_def.ParseFromString(serialized_graph)
 
        tf.import_graph_def(od_graph_def, name = "")
 
def run_inference_for_single_image(image, graph):
    with tf.compat.v1.Session(graph = graph) as sess:
 
        input_tensor = graph.get_tensor_by_name('image_tensor:0')
        
        target_operation_names = ['num_detections', 'detection_boxes',
                                  'detection_scores', 'detection_classes', 'detection_masks']
        tensor_dict = {}
        for key in target_operation_names:
            op = None
            try:
                op = graph.get_operation_by_name(key)
                
            except:
                continue
 
            tensor = graph.get_tensor_by_name(op.outputs[0].name)
            tensor_dict[key] = tensor
 
        if 'detection_masks' in tensor_dict:
            detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
 
        output_dict = sess.run(tensor_dict, feed_dict = {input_tensor : [image]})
            
        output_dict['num_detections'] = int(output_dict['num_detections'][0])
        output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
        output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
        output_dict['detection_scores'] = output_dict['detection_scores'][0]
 
        return output_dict
   
PATH_TO_TEST_IMAGES_DIR = 'data/images_choice/train/jpgs'
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image_{}.jpg'.format(i)) for i in range(1,41) ]

cnt = 0

for image_path in TEST_IMAGE_PATHS: 
    cnt += 1  
    print(cnt) 
    image_np = cv2.imread(image_path)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    height, width, _ = image_np.shape
    
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    draw_bounding_boxes(image_np, output_dict, cnt)
